In [ ]:
%pip install enformer-pytorch

In [ ]:
import torch
import numpy as np
from Bio import SeqIO
from enformer_pytorch import Enformer

In [ ]:
!gunzip ./Homo_sapiens.GRCh38.113.gtf.gz
!gunzip ./Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

In [ ]:
!head -6 Homo_sapiens.GRCh38.113.gtf | tail -1

In [ ]:
!tail -50 Homo_sapiens.GRCh38.dna.primary_assembly.fa

In [ ]:
#This extracts gene regions and saves them in a BED file format (chrom start end gene_info).

!awk '$3 == "gene" {match($0, /gene_id "([^"]+)"/, id); print "chr"$1 "\t" $4-1 "\t" $5 "\t" id[1] "\t.\t" $7}' Homo_sapiens.GRCh38.113.gtf > genes.bed

In [ ]:
!grep "ENSG00000012048" genes.bed > BRCA1.bed

In [ ]:
conda install -c bioconda

In [ ]:
# Load the genome FASTA into a dictionary
fasta_file = "Homo_sapiens.GRCh38.dna.primary_assembly.fa"
genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

# Read BED file
bed_file = "BRCA1.bed"
output_file = "BRCA1_genes.fa"

with open(bed_file, "r") as bed, open(output_file, "w") as output:
    for line in bed:
        fields = line.strip().split("\t")
        chrom, start, end, gene_id, score, strand = fields
        
        start, end = int(start), int(end)
        
        if chrom in genome:
            seq = genome[chrom].seq[start:end]  # Extract sequence
            if strand == "-":  # Reverse complement if on negative strand
                seq = seq.reverse_complement()
            
            output.write(f">{gene_id} {chrom}:{start}-{end}({strand})\n{seq}\n")

print(f"Extracted sequences saved to {output_file}")

In [ ]:
# Now to perform one hot encoding 

In [ ]:
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 
               'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'N': [0, 0, 0, 0]}
    return np.array([mapping[base] for base in seq], dtype=np.uint8)

# Load extracted sequences
fasta_file = "BRCA1_genes.fa"
sequences = []

for record in SeqIO.parse(fasta_file, "fasta"):
    seq = str(record.seq).upper()
    if len(seq) >= 196608:  # Trim or pad to Enformer input size
        seq = seq[:196608]
    else:
        seq += "N" * (196608 - len(seq))  # Pad with Ns

    one_hot_seq = one_hot_encode(seq)
    sequences.append(one_hot_seq)

sequences = np.array(sequences)  # Convert to NumPy array
np.save("one_hot_sequences_BRCA1_4.npy", sequences)  # Save for later use

print("One-hot encoded sequences saved as one_hot_sequences_BRCA1_4.npy")


In [ ]:
torch.cuda.empty_cache()

# Load saved one-hot encoded sequences
data = np.load("one_hot_sequences_BRCA1_4.npy")  # Shape: (batch_size, 196608, 4)

# Convert to PyTorch tensor & reorder dimensions
one_hot_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 1, 2)  # (batch, 4, 196608)

# Load Enformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Enformer.from_hparams(
    dim=1536,
    depth=11,
    heads=8,
    output_heads={"human": 5313},  
    target_length=896
).to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
one_hot_tensor = one_hot_tensor.to(device)  # Move tensor to GPU

# Reduce batch size
batch_size = 30  # Adjust based on available memory
num_samples = one_hot_tensor.shape[0]

model.eval()

with torch.no_grad():
    outputs = []
    for i in range(0, num_samples, batch_size):
        batch = one_hot_tensor[i:i+batch_size]  # Take a batch of sequences
        output = model(batch)  # Forward pass
        outputs.append(output["human"].cpu().numpy())  # Store predictions

# Combine results
human_predictions = np.concatenate(outputs, axis=0)

# Save predictions
np.save("enformer_human_predictions.npy", human_predictions)
print("Predictions saved successfully!")

In [ ]:
# Load the .npy file
data = np.load("enformer_human_predictions.npy")

# Check its shape and type
print("Shape:", data.shape)
print("Data type:", data.dtype)

# Small portion of the data
print(data[:5])  # First 5 elements (adjust based on your data)
